In [1]:
from datasets import load_dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [2]:
dataset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})

In [3]:
def format_row(example):
  question = example['Context']
  answer = example['Response']
  example['Text'] = f"[INST] {question} [/INST] {answer}"

  return example

In [4]:
formatted_dataset = dataset.map(format_row)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [5]:
formatted_dataset

Dataset({
    features: ['Context', 'Response', 'Text'],
    num_rows: 3512
})

In [6]:
print(formatted_dataset[0]['Text'])

[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad fee

In [7]:
import pandas as pd

# convert dataset to DataFrame
df = pd.DataFrame(dataset)

In [8]:
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [9]:
import random

INSTRUCTIONS = [
    "Respond empathetically and helpfully to the user's emotional concern.",
    "Offer emotional support and practical guidance to the user.",
    "Provide a compassionate and constructive response."
]

def convert_with_variation(example):
    return {
        "instruction": random.choice(INSTRUCTIONS),
        "input": example["Context"].strip(),
        "output": example["Response"].strip(),
    }


In [10]:
ready_dataset = dataset.map(
    convert_with_variation,
    remove_columns=dataset.column_names
)


Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [11]:
ready_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3512
})

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [13]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [15]:
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

In [16]:
def format_prompt(example):
    prompt = (
        f"<s>[INST] {example['instruction']}\n\n"
        f"{example['input']} [/INST] "
        f"{example['output']}</s>"
    )
    return {"text" : prompt}


In [17]:
ready_dataset = ready_dataset.map(format_prompt)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [18]:
ready_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 3512
})

In [19]:
print(ready_dataset[0]['text'])

<s>[INST] Offer emotional support and practical guidance to the user.

I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't 

In [20]:
def tokenize_func(example):
  tokens = tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)
  tokens['labels'] = tokens['input_ids'].copy()
  return tokens

In [21]:
tokenized_dataset = ready_dataset.map(tokenize_func, batched=True)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [22]:
lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ['q_proj', 'v_proj'],
    bias='none'
)

In [23]:
import torch

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1566: UserWarning: Current model requires 64 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [45]:
model = get_peft_model(model, lora_config)


In [46]:
args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.0,
    logging_steps=50,
    save_total_limit=1,
    report_to="none",
)


In [47]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_dataset
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [48]:
trainer.train()

Step,Training Loss
50,2.864600
100,1.474600
150,1.384800
200,1.354800
250,1.318900
300,1.281600
350,1.315600
400,1.286600
450,1.275400
500,1.288500


TrainOutput(global_step=1317, training_loss=1.3496535669788774, metrics={'train_runtime': 2010.8961, 'train_samples_per_second': 5.239, 'train_steps_per_second': 0.655, 'total_flos': 3.352009798110413e+16, 'train_loss': 1.3496535669788774, 'epoch': 3.0})

In [59]:
prompt = "Some days I feel like I’m barely holding myself together and one small thing could make me break."

In [60]:
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

In [61]:
outputs = model.generate(
    **inputs,
    max_new_tokens = 256,
    temperature = 0.8,
    top_p = 0.9,
    do_sample = True,
    repetition_penalty = 1.1
)

In [62]:
print("/n Model Output /n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/n Model Output /n
Some days I feel like I’m barely holding myself together and one small thing could make me break.

I always find a way to be happy even when things are not perfect. There will never be a day where I’m not proud of myself or grateful for what I have.

In the midst of this, there is someone who loves me so much that he has no idea how I am feeling every single minute. That’s the kind of love that will always bring me joy. He is my rock and my anchor. He has been there through the highs and lows, through everything I’ve ever wanted and dreamed of, including how I would like him to feel about me.

My family and friends have been there for me in times of need, and I know they will always be there when I need them too. It takes a lot to see someone who knows you better than anyone else, but I’ve found that to be true. People can tell if someone doesn’t care about them, but it’s more difficult for people to tell if someone cares deeply about them.

It takes courage to speak